In [4]:
import pandas as pd
import numpy as np

path = 'data/sts_benchmark/'

TRAIN_DATA_FILE = path + 'sts-train.csv'
TEST_DATA_FILE = path + 'sts-test.csv'

train_df = pd.read_csv(TRAIN_DATA_FILE, '\t', error_bad_lines=False)
test_df = pd.read_csv(TEST_DATA_FILE, '\t', quoting=3, error_bad_lines=False)

test_df.head()

b'Skipping line 2508: expected 7 fields, saw 8\nSkipping line 2649: expected 7 fields, saw 8\nSkipping line 2726: expected 7 fields, saw 8\nSkipping line 3070: expected 7 fields, saw 8\nSkipping line 3392: expected 7 fields, saw 8\nSkipping line 5515: expected 7 fields, saw 9\nSkipping line 5516: expected 7 fields, saw 9\nSkipping line 5517: expected 7 fields, saw 9\nSkipping line 5518: expected 7 fields, saw 9\nSkipping line 5519: expected 7 fields, saw 9\nSkipping line 5520: expected 7 fields, saw 9\nSkipping line 5521: expected 7 fields, saw 9\nSkipping line 5522: expected 7 fields, saw 9\nSkipping line 5523: expected 7 fields, saw 9\nSkipping line 5524: expected 7 fields, saw 9\nSkipping line 5525: expected 7 fields, saw 9\nSkipping line 5526: expected 7 fields, saw 9\nSkipping line 5527: expected 7 fields, saw 9\nSkipping line 5528: expected 7 fields, saw 9\nSkipping line 5529: expected 7 fields, saw 9\nSkipping line 5530: expected 7 fields, saw 9\nSkipping line 5531: expected 7 f

,main-captions,MSRvid,2012test,0024,2.500,A girl is styling her hair.,A girl is brushing her hair.
0,main-captions,MSRvid,2012test,33,3.6,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.
1,main-captions,MSRvid,2012test,45,5.0,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.
2,main-captions,MSRvid,2012test,63,4.2,A man is cutting up a cucumber.,A man is slicing a cucumber.
3,main-captions,MSRvid,2012test,66,1.5,A man is playing a harp.,A man is playing a keyboard.
4,main-captions,MSRvid,2012test,74,1.8,A woman is cutting onions.,A woman is cutting tofu.


In [24]:
import os
from collections import Counter

from gensim.models import Word2Vec
from nltk.tokenize import WordPunctTokenizer
import numpy as np
# from keras import Input, Model
# from keras.callbacks import ModelCheckpoint
# from keras.layers import Embedding, MaxPooling1D, multiply, subtract, Lambda, concatenate, Dense, Conv1D, Flatten, \
#     Reshape
from tqdm import tqdm

sent1_file_name = 'data/sts_benchmark/sent1.txt'
sent2_file_name = 'data/sts_benchmark/sent2.txt'
sent1_test_file_name = 'data/sts_benchmark/sent1_test.txt'
sent2_test_file_name = 'data/sts_benchmark/sent2_test.txt'


def content_provider(file_name):
    with open(file_name) as file:
        content = file.readlines()
        content = [line.split('\n')[0] for line in content]
        return content


sentences1 = content_provider(sent1_file_name)
sentences2 = content_provider(sent2_file_name)
sentences1_test = content_provider(sent1_test_file_name)
sentences2_test = content_provider(sent2_test_file_name)

tokenizer = WordPunctTokenizer()
vocab = Counter()


def text_to_wordlist(text, lower=False):
    text = tokenizer.tokenize(text)
    if lower:
        text = [t.lower() for t in text]
    vocab.update(text)
    return text


def process_sentences(list_sentences, lower=False):
    words = []
    for text in tqdm(list_sentences):
        txt = text_to_wordlist(text, lower=lower)
        words.append(txt)
    return words


all_sentences = sentences1 + sentences2 + sentences1_test + sentences2_test
words = process_sentences(all_sentences, lower=True)

print("The vocabulary contains {} unique tokens".format(len(vocab)))

model = Word2Vec(words, size=300, window=5, min_count=5, workers=8, sg=0, negative=5)
word_vectors = model.wv
print("Number of word vectors: {}".format(len(word_vectors.vocab)))
model.wv.most_similar_cosmul(positive=['she', 'man'], negative=['he'])

[('staring', 1.3314028978347778),
 ('yard', 1.287097692489624),
 ('sweater', 1.2835159301757812),
 ('riding', 1.2610979080200195),
 ('standing', 1.257534384727478),
 ('woman', 1.2561559677124023),
 ('coat', 1.2551298141479492),
 ('singing', 1.253724455833435),
 ('horse', 1.251882791519165),
 ('playing', 1.2514890432357788)]

In [ ]:
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 30

word_index = {t[0]: i+1 for i, t in enumerate(vocab.most_common(MAX_NB_WORDS))}
print(word_index)
sequences = [[word_index.get(t, 0) for t in word]
             for word in words[:len(list_sentences_train)]]
test_sequences = [[word_index.get(t, 0)  for t in comment] 
                  for comment in comments[len(list_sentences_train):]]

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="post", truncating="post", value=0.0)
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre",
                          truncating="post")
print('Shape of test_data tensor:', test_data.shape)
